<a href="https://colab.research.google.com/github/Ruijia0/SupplyChain_Analytics/blob/main/Tailored_sourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import math

# Product data
products = {
    1: {"D": 1000, "A_c": 150, "A_s": 20, "c": 50, "h": 0.15},
    2: {"D": 300,  "A_c": 150, "A_s": 25, "c": 60, "h": 0.15},
    3: {"D": 100,  "A_c": 150, "A_s": 30, "c": 30, "h": 0.15},
    4: {"D": 50,   "A_c": 150, "A_s": 50, "c": 30, "h": 0.15}
}

# Calculate holding cost multiplier and order cost for independent strategy
def calculate_costs():
    # Scenario 1: Independent Sourcing
    total_independent_cost = 0
    for p in products.values():
        Q = math.sqrt((2 * p["D"] * (p["A_c"] + p["A_s"])) / (p["c"] * p["h"]))
        ordering_cost = p["D"] / Q * (p["A_c"] + p["A_s"])
        holding_cost = Q / 2 * p["c"] * p["h"]
        total_cost = ordering_cost + holding_cost
        total_independent_cost += total_cost

    # Scenario 2: Joint Sourcing with Same Frequency
    sum_A = sum(p["A_s"] for p in products.values()) + products[1]["A_c"]  # Common cost counted once
    sum_S = sum((p["D"] * p["c"] * p["h"] / 2) for p in products.values())
    n = math.sqrt(sum_S / sum_A)
    total_joint_cost = sum_A * n + sum_S / n

    # Scenario 3: Tailored Aggregation (simplified calculation)
    # Calculate frequencies using specific costs only for tailored aggregation
    freqs = {id: math.sqrt((2 * prod["D"] * prod["A_s"]) / (prod["c"] * prod["h"])) for id, prod in products.items()}
    most_freq_product = min(freqs, key=freqs.get)
    most_freq_rate = freqs[most_freq_product]

    # Adjust frequencies based on the most frequently ordered product
    multiples = {id: math.ceil(most_freq_rate / freq) for id, freq in freqs.items() if id != most_freq_product}
    multiples[most_freq_product] = 1  # The most frequently ordered product itself

    # Apply a less aggressive frequency adjustment
    adjusted_freq = most_freq_rate / (1 + sum(1/mult for mult in multiples.values()) * 0.9)  # 90% efficiency factor

    # Calculate total cost accounting for less optimal common cost sharing
    total_aggregated_cost = 0
    for id, prod in products.items():
        freq = adjusted_freq * multiples[id]
        Q = prod["D"] / freq
        specific_order_cost = prod["A_s"] * freq
        # Apply common cost proportionally to how often the product orders relative to the most frequent product
        common_order_cost = prod["A_c"] * freq / multiples[id]  # Less frequent application of common cost
        holding_cost = (Q / 2) * prod["c"] * prod["h"]
        total_aggregated_cost += specific_order_cost + common_order_cost + holding_cost

    return total_independent_cost, total_joint_cost, total_aggregated_cost

# Calculate and display results
cost_independent, cost_joint, cost_aggregated = calculate_costs()
print(f"Independent Sourcing Total Cost: {cost_independent:.2f}")
print(f"Joint Sourcing Total Cost: {cost_joint:.2f}")
print(f"Tailored Aggregation Sourcing Total Cost: {cost_aggregated:.2f}")


Independent Sourcing Total Cost: 3271.48
Joint Sourcing Total Cost: 2445.66
Tailored Aggregation Sourcing Total Cost: 6004.00
